# Purpose of notebook
This notebook examplifies the replication of the shapleyPermEx and shapleyPermRand functions from the R package 'sensitivity'. I implement the functions to see if we get the same results as the R package in order to consequently apply the methods to the respy model.

In [1]:
import itertools

import numpy as np
import pandas as pd
import chaospy as cp
import openturns as ot

from econsa.sampling import cond_mvn

## Shapley function

Code of original R package:

- [sensitivity source: R/shapleyPermEx.R](https://rdrr.io/cran/sensitivity/src/R/shapleyPermEx.R)
- [sensitivity source: R/shapleyPermRand.R](https://rdrr.io/cran/sensitivity/src/R/shapleyPermRand.R)

In [120]:
# define shapley function as in the R package
def ShapleyPerm(method, m, model, moyenne, cov, d, Nv, No, Ni=3):
    """
    """
    if (method == 'exact'):
        perms = list(itertools.permutations(range(d), d))
        perms = [list(i) for i in perms]
        m = len(perms)
    else:
        perms = np.zeros((m,d), dtype = np.int64)
        for i in range(m):
            perms[i] = np.random.permutation(d)

    #------------------------------
    # Creation of the design matrix
    #------------------------------

    X = np.zeros((Nv + m * (d - 1) * No * Ni, d)) 
    X[:Nv, :] = Xall(moyenne, cov, Nv).T

    for p in range(m):
    
        pi = perms[p]
        pi_sorted = np.argsort(pi)
    
        for j in range(1,d):
            # set of the 0st-(j-1)th elements in pi
            Sj = pi[:j]
            # set of the jth-dth elements in pi
            Sjc = pi[j:]
        
            # sampled values of the inputs in Sjc
            xjcM = np.matrix(Xcond(moyenne, cov, No, Sjc, None, None)).T
#             print(xjcM.shape)
            
            for l in range(No):
#                 print("m =", m)
#                 print("l =", l)
                xjc = xjcM[l, :]
            
#                 print("Sjc =", Sjc)
#                 print("xjc =", xjc)
                # sample values of inputs in Sj conditional on xjc
                xj = np.matrix(Xcond(moyenne, cov, Ni, Sj, Sjc, xjc.flat)).T
                xx = np.concatenate((xj, np.ones((Ni, 1)) * xjc), axis = 1).T
                ind_inner = Nv + p * (d - 1) * No * Ni + (j - 1) * No * Ni + l * Ni
                X[ind_inner:(ind_inner + Ni), :] = xx[:, pi_sorted]
    
    #-----------------------
    # Calcul of the response
    #-----------------------
    
    y = model(X)

    #-----------------------------------------------------------------
    # Initialize Shapley, main and total Sobol effects for all players
    #-----------------------------------------------------------------
    
    Sh = np.zeros(d)
    Vsob = np.zeros(d)
    Tsob = np.zeros(d)
    
    nV = np.zeros(d) # number of samples used to estimate V1,...,Vd
    nT = np.zeros(d) # number of samples used to estimate T1,...,Td
    
    #----------------
    # Estimate Var[Y]
    #----------------
    
    Y = y[:Nv]
    y = y[Nv:]
    EY = np.mean(Y)
    VarY = np.var(Y)

    #-----------------------------------------------
    # Estimate Shapley, main and total Sobol effects
    #-----------------------------------------------
    
    cVar = np.zeros(No)

    for p in range(m):
    
        pi = perms[p]
        prevC = 0
    
        for j in range(d):
            if (j == (d - 1)):
                Chat = VarY
                delta = Chat - prevC
                Vsob[pi[j]] = Vsob[pi[j]] + prevC # first order effect
                nV[pi[j]] = nV[pi[j]] + 1
            else:
                for l in range(No):
                    Y = y[:Ni]
                    y = y[Ni:]
                    cVar[l] = np.var(Y)
                Chat = np.mean(cVar)
                delta = Chat - prevC
      
            Sh[pi[j]] = Sh[pi[j]] + delta
        
            prevC = Chat
        
            if (j == 0):
                Tsob[pi[j]] = Tsob[pi[j]] + Chat # Total effect
                nT[pi[j]] = nT[pi[j]] + 1
    
    Sh = Sh / m / VarY
    
    if (method == 'exact'):
        Vsob = Vsob / (m/d) / VarY # averaging by number of permutations with j=d-1
        Vsob = 1 - Vsob 
        Tsob = Tsob / (m/d) / VarY # averaging by number of permutations with j=1 
    else:
        Vsob = Vsob / nV / VarY # averaging by number of permutations with j=d-1
        Vsob = 1 - Vsob 
        Tsob = Tsob / nT / VarY # averaging by number of permutations with j=1 
    
    col = ['X' + str(i) for i in np.arange(d) + 1]
    effects = pd.DataFrame(
        np.array([Sh, Vsob, Tsob]),
        index = ['Shapley effects', 'First order Sobol', 'Total Sobol'],
        columns = col,
    )

    return effects

In [10]:
def gaussian_model(X):
    return np.sum(X,1)

In [110]:
def Xall(moyenne, cov, n):
    distribution = cp.MvNormal(moyenne, cov)
    return distribution.sample(n)

In [117]:
def Xcond(moyenne, cov, n, Sj, Sjc, xjc):
    if Sjc is None:
        cov_int = np.array(cov)
        cov_int = cov_int.take(Sj, axis = 1)
        cov_int = cov_int[Sj]
        distribution = cp.MvNormal(moyenne[Sj], cov_int)
        return distribution.sample(n)
    else:
        return r_condMVN(n,mean = moyenne, cov = cov, dependent_ind = Sj, given_ind = Sjc, X_given = xjc)

# Conditional sampling functions
These funcitons estimate the conditional vector for use in the shapley function

In [118]:
# Generate conditional law
def r_condMVN(n, mean, cov, dependent_ind, given_ind, X_given):
    """ Function to simulate conditional gaussian distribution of X[dependent.ind] | X[given.ind] = X.given
    where X is multivariateNormal(mean = mean, covariance = cov)"""
    cond_mean,cond_var = cond_mvn(
        mean, cov, dependent_ind = dependent_ind, given_ind = given_ind, given_value = X_given,
    )
    distribution = cp.MvNormal(cond_mean, cond_var)
    
    return distribution.sample(n)

## Evaluation Shapley effects on linear test model model

In [121]:
# Ni = 3

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])

# Exact method
method = 'exact'
m = None
Nv = 10 ** 4
No = 10 ** 3
Ni = 3

np.random.seed(1)
ot.RandomGenerator.SetSeed(0)
index = ShapleyPerm(method, m, gaussian_model, moyenne, cov, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

Exact method 
                         X1        X2        X3
Shapley effects    0.371839  0.338162  0.289999
First order Sobol  0.563545  0.355004  0.315657
Total Sobol        0.721465  0.837247  0.787473




In [97]:
# Random method
method = 'random'
m = 6000
Nv = 10 ** 4
No = 1
Ni = 3

ot.RandomGenerator.SetSeed(0)
index = ShapleyPerm(method, m, gaussian_model, moyenne, cov, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')

Random method 
                         X1        X2        X3
Shapley effects    0.363450  0.363483  0.273066
First order Sobol  0.579954  0.414898  0.266531
Total Sobol        0.697817  0.850366  0.781962



In [39]:
# Ni = 100

d = 3
moyenne = np.zeros(3)
cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
cov = ot.CovarianceMatrix(cov)

# Exact method
method = 'exact'
m = None
Nv = 10 ** 4
No = 10 ** 3
Ni = 100

ot.RandomGenerator.SetSeed(0)
index = ShapleyPerm(method,m,gaussian_model, moyenne, cov, d, Nv, No, Ni)
print('Exact method \n' + str(index) + '\n\n')

Exact method 
                         X1        X2        X3
Shapley effects    0.107076  0.416297  0.476628
First order Sobol  0.113784  0.817406  0.877446
Total Sobol        0.104018  0.019542  0.078596




In [41]:
# Random method
method = 'random'
m = 6000
Nv = 10 ** 4
No = 1
Ni = 100

ot.RandomGenerator.SetSeed(0)
index = ShapleyPerm(method,m,gaussian_model, moyenne, cov, d, Nv, No, Ni)
print('Random method \n' + str(index) + '\n')

Random method 
                         X1        X2        X3
Shapley effects    0.105530  0.413029  0.481441
First order Sobol  0.110748  0.818254  0.877002
Total Sobol        0.103103  0.019625  0.078566

